## COMUNICACIÓN EN TIEMPO REAL CON EL ROBOT

In [2]:
import cv2
import numpy as np
import serial
import time

def detect_shape(frame, cnt):
    epsilon = 0.04 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    sides = len(approx)

    if sides == 3:
        return "TRIANGULO", b'1'
    elif sides == 4:
        x, y, w, h = cv2.boundingRect(approx)
        aspect_ratio = float(w) / h
        if abs(1 - aspect_ratio) < 0.05:
            return "CUADRADO", b'4'
        else:
            return "RECTANGULO", b'2'
    elif sides == 5:  # Check for a pentagon
        return "PENTAGONO", b'5'
    else:
        x, y, w, h = cv2.boundingRect(cnt)
        roi = frame[y:y+h, x:x+w]
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        return "NINGUNA", b'0'

def main():
    arduino = serial.Serial("COM3", 9600)
    time.sleep(0)

    font = cv2.FONT_HERSHEY_COMPLEX

    cap = cv2.VideoCapture(1)

    last_send_time = time.time()  # Initialize the last send time

    while True:
        _, frame = cap.read()
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower_green = np.array([60, 60, 60])
        upper_green = np.array([180, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        figura, valor = "NINGUNA", b'0'
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 400:
                cv2.drawContours(frame, [cnt], 0, (0, 0, 255), 2)
                figura, valor = detect_shape(frame, cnt)

        cv2.putText(frame, "FORMA DETECTADA: ", (20, 30), font, 1, (255, 0, 0), 3)
        cv2.putText(frame, figura, (220, 70), font, 1, (0, 0, 255), 3)
        cv2.imshow("Original", frame)
        cv2.imshow("Mascara", mask)

        # Check if 10 seconds have passed since the last value was sent
        if time.time() - last_send_time >= 5:
            arduino.write(valor)
            print(valor)
            last_send_time = time.time()  # Update the last send time

        key = cv2.waitKey(1)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

    try:
        arduino.close()
    except:
        pass

if __name__ == "__main__":
    main()

b'0'
b'0'
b'0'
b'0'
b'0'
b'5'
b'5'
b'0'
b'0'
b'0'
b'0'
b'0'
b'0'
b'5'
b'5'
b'0'
b'0'
b'0'
b'0'
b'0'
b'0'
b'0'
b'0'
b'5'
b'5'
b'0'
b'0'
b'2'
b'1'
b'0'
b'0'
b'0'
b'0'
b'0'
b'1'
b'1'
b'1'
b'1'
b'1'
b'1'
b'1'
b'0'
b'0'
b'1'
b'1'
b'0'
b'0'
b'0'
b'2'
b'2'
b'5'
b'5'
b'5'
b'5'
b'5'
b'5'
b'5'
b'0'
b'0'
b'0'
b'0'
b'0'
b'5'
b'5'
b'0'
b'0'
b'0'
b'0'
b'2'
b'0'
b'0'
b'0'
b'0'
b'5'
b'5'
b'5'
b'5'
b'0'
b'5'
b'5'
b'0'
b'0'
b'0'
b'0'
b'0'
b'5'
b'5'
b'5'
b'5'
b'0'
b'0'
b'5'
b'5'
b'0'
b'5'
b'0'
b'0'
b'0'
b'0'
b'5'
b'5'
b'5'
b'5'
b'5'
b'5'
b'5'
b'5'
b'0'
b'0'
b'0'
b'0'
b'0'
b'0'
b'5'
b'5'
b'0'
b'0'
b'1'
b'1'
b'0'
b'0'
b'0'
b'2'
b'0'
b'2'
b'2'
b'2'
b'2'
b'2'
b'2'
b'0'
b'0'
b'0'
b'0'
b'0'
b'0'


## CALIBRACIÓN DEL HSV (CONTORNO Y COLOR DE LA FIGURA A DETECTAR)

In [8]:
import cv2
import numpy as np
import serial
import time

arduino = serial.Serial("COM3", 9600)
time.sleep(2)

def detect_shape(cnt):
    epsilon = 0.02 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    sides = len(approx)

    if sides == 3:
        return "TRIANGULO", b'1'
    elif sides == 4:
        x, y, w, h = cv2.boundingRect(approx)
        aspect_ratio = float(w) / h if h != 0 else 0
        if abs(1 - aspect_ratio) < 0.2:
            return "CUADRADO", b'4'
        else:
            return "RECTANGULO", b'2'
    elif 7 < sides < 20:
        return "CIRCULO", b'3'
    else:
        return "NINGUNA", b'0'

def calibrate_color(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    cv2.namedWindow("Calibracion")
    cv2.createTrackbar("L-H", "Calibracion", 60, 180, lambda x: None)
    cv2.createTrackbar("L-S", "Calibracion", 100, 255, lambda x: None)
    cv2.createTrackbar("L-V", "Calibracion", 60, 255, lambda x: None)
    cv2.createTrackbar("U-H", "Calibracion", 180, 180, lambda x: None)
    cv2.createTrackbar("U-S", "Calibracion", 255, 255, lambda x: None)
    cv2.createTrackbar("U-V", "Calibracion", 255, 255, lambda x: None)

    while True:
        l_h = cv2.getTrackbarPos("L-H", "Calibracion")
        l_s = cv2.getTrackbarPos("L-S", "Calibracion")
        l_v = cv2.getTrackbarPos("L-V", "Calibracion")
        u_h = cv2.getTrackbarPos("U-H", "Calibracion")
        u_s = cv2.getTrackbarPos("U-S", "Calibracion")
        u_v = cv2.getTrackbarPos("U-V", "Calibracion")

        lower_green = np.array([l_h, l_s, l_v])
        upper_green = np.array([u_h, u_s, u_v])

        mask = cv2.inRange(hsv, lower_green, upper_green)
        result = cv2.bitwise_and(frame, frame, mask=mask)

        cv2.imshow("Calibracion", result)

        key = cv2.waitKey(1)
        if key == 27:
            cv2.destroyAllWindows()
            break

    return lower_green, upper_green

cap = cv2.VideoCapture(1)

_, frame = cap.read()
lower_green, upper_green = calibrate_color(frame)

font = cv2.FONT_HERSHEY_COMPLEX
valor = b'0'
listo = b'0'
figura = "NINGUNA"

while True:
    _, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, lower_green, upper_green)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel)
    mask = cv2.dilate(mask, kernel)

    if arduino.inWaiting():
        listo = arduino.read()
        print("llego ", listo)

        # Contours detection
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 400:
                cv2.drawContours(frame, [cnt], 0, (0, 0, 255), 4)
                figura, valor = detect_shape(cnt)

        if listo == b'2':
            print("====entro ", valor)
            arduino.write(valor)
            valor = b'0'
            listo = b'0'

    cv2.putText(frame, "FORMA DETECTADA: ", (20, 30), font, 1, (255, 0, 0), 3)
    cv2.putText(frame, figura, (220, 70), font, 1, (0, 0, 255), 3)
    cv2.imshow("Original", frame)
    cv2.imshow("Mascara", mask)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()
arduino.close()

